# Submitting (uploading) data to the JCOIN Data Commons

To upload (and update) a core measure (or other package or file for that matter) to the JDC, one can use the `submit_package_to_jdc` and `map_to_sheepdog` functions (in this case, we use `map_core_measures_to_sheepdog` which runs a set of transforms to go from a validated core measure data package to a sheepdog submission and then uses `map_to_sheepdog` in the `jdc_utils.utils.gen3` module). Sheepdog maps a subset of variables for quicker discovery on the JDC.


## Create a new file record (does have any old version- entirely new file)

> This is commented out as a TEST project file exists

In [1]:
# from jdc_utils.submission import submit_package_to_jdc,map_core_measures_to_sheepdog
# from frictionless import Package
# import os

In [4]:
# from gen3.auth import Gen3Auth
# from gen3.submission import Gen3Submission
# from gen3.index import Gen3Index
# from gen3.file import Gen3File
# index = Gen3Index(Gen3Auth(refresh_file="credentials.json"))
# sub = Gen3Submission(Gen3Auth(refresh_file="credentials.json"))
# files = Gen3File(Gen3Auth(refresh_file="credentials.json"))
# sheepdog_rec = sub.export_node("JCOIN","TEST","reference_file",fileformat="json")
# index_rec = index.get_record(sheepdog_rec["data"][-1]["object_id"])
## files.delete_file_locations(index_rec["object_id"])
## sub.delete_record("JCOIN","TEST",sheepdog_rec["data"][-1]["id"])

In [ ]:
# # create new record - do this only one  time!
# # NOTE: no file guid because the record has not been created yet
# # NOTE: if want to test with new file records, delete the ones added by
#  # uncommenting the deletion methods in above cell
# gen3_file = submit_package_to_jdc(
#     package_path="../data/core-measures-test.zip",
#     commons_project=commons_project,
#     sheepdog_file_submitter_id='core-measures-TEST',
#     sheepdog_data_type="Interview",
#     sheepdog_data_category="Core Measures",
#     sheepdog_data_format="ZIP",
#     submission_type="create",
# )

Save the metadata to a file for later reference when updating file

In [30]:
#TODO

## Map variables to sheepdog 
This uses the specific `map_core_measures_to_sheepdog` which calls the 
more general `map_to_sheepdog` function

In [7]:
# package_path = "../data/core-measures-test/data-package.json"
# core_measures = Package(package_path)
# baseline_df = core_measures.get_resource("baseline").to_pandas()
# baseline_df["role_in_project"] = "Client"
# timepoints_df = core_measures.get_resource("timepoints").to_pandas()
# last_node_output = map_core_measures_to_sheepdog(
#     baseline_df=baseline_df,
#     timepoints_df=timepoints_df,
#     program="JCOIN",
#     project="TEST",
#     credentials_path="credentials.json"
# )

Validating records for participant
Validating records for enrollment
Validating records for demographic_baseline
Validating records for time_point
Deleting records for time_point

Output written to file: tmp/sheepdog/time_point.json
time_point.
Deleting records for demographic_baseline

Output written to file: tmp/sheepdog/demographic_baseline.json
demographic_baseline.
Deleting records for enrollment

Output written to file: tmp/sheepdog/enrollment.json
enrollment.
Deleting records for participant

Output written to file: tmp/sheepdog/participant.json
participant.
Submitting records for participant
[2023-04-12 11:26:26,734][   INFO] 
Using the Sheepdog API URL https://jcoin.datacommons.io/api/v0/submission/JCOIN/TEST

Submitting records for enrollment
[2023-04-12 11:26:27,543][   INFO] 
Using the Sheepdog API URL https://jcoin.datacommons.io/api/v0/submission/JCOIN/TEST

Submitting records for demographic_baseline
[2023-04-12 11:26:28,325][   INFO] 
Using the Sheepdog API URL https://

## Update file record

To simulate an update: Change something in package, rezip, and run upload/sheepdog mapping

In [1]:
from jdc_utils.submission import CoreMeasures

commons_project = "TEST"
file_guid = 'dg.6VTS/80b0372c-a90f-44f9-87a4-c699ed11d051'
sheepdog_file_submitter_id = 'core-measures-test'
credentials_path="credentials.json"

In [2]:
# make slightly updated zipped file
core_measures = CoreMeasures("../data/external_workflow_slight_update",is_core_measures=True)
core_measures.write(outdir="../data/core-measures-slight-update-test")

/Users/michaelkranz/Documents/norc-projects/jcoin/jdc-utilities/notebooks
/Users/michaelkranz/Documents/norc-projects/jcoin/jdc-utilities/data/external_workflow_slight_update


/Users/michaelkranz/anaconda/envs/jdc-utilities/lib/python3.10/site-packages/frictionless/plugins/pandas/parser.py:64: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, dtype in dataframe.dtypes.iteritems():
/Users/michaelkranz/anaconda/envs/jdc-utilities/lib/python3.10/site-packages/frictionless/plugins/pandas/parser.py:64: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, dtype in dataframe.dtypes.iteritems():
/Users/michaelkranz/anaconda/envs/jdc-utilities/lib/python3.10/site-packages/frictionless/plugins/pandas/parser.py:64: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, dtype in dataframe.dtypes.iteritems():
/Users/michaelkranz/anaconda/envs/jdc-utilities/lib/python3.10/site-packages/frictionless/plugins/pandas/parser.py:64: FutureWarning: iteritems is deprecated and will be removed in a future 

In [4]:
core_measures.submit(
    commons_project_code=commons_project,
    commons_file_guid=file_guid,
    commons_file_submitter_id=sheepdog_file_submitter_id,
    commons_credentials_path=credentials_path
)

The latest sheepdog and indexd have same file:GOOD
  Submitting 12 records in batches of 100
Submission progress: 1/1


/Users/michaelkranz/anaconda/envs/jdc-utilities/lib/python3.10/site-packages/frictionless/plugins/pandas/parser.py:64: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, dtype in dataframe.dtypes.iteritems():
/Users/michaelkranz/anaconda/envs/jdc-utilities/lib/python3.10/site-packages/frictionless/plugins/pandas/parser.py:64: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, dtype in dataframe.dtypes.iteritems():


Validating records for participant
Validating records for enrollment
Validating records for demographic_baseline
Validating records for time_point
Deleting records for time_point

Output written to file: tmp/sheepdog/time_point.json
time_point.
Deleting records for demographic_baseline

Output written to file: tmp/sheepdog/demographic_baseline.json
demographic_baseline.
Deleting records for enrollment

Output written to file: tmp/sheepdog/enrollment.json
enrollment.
Deleting records for participant

Output written to file: tmp/sheepdog/participant.json
participant.
Submitting records for participant
  Submitting 4 records in batches of 100
Submission progress: 1/1
Submitting records for enrollment
  Submitting 4 records in batches of 100
Submission progress: 1/1
Submitting records for demographic_baseline
  Submitting 4 records in batches of 100
Submission progress: 1/1
Submitting records for time_point
  Submitting 11 records in batches of 100
Submission progress: 1/1
